In [1]:
import json

In [2]:
path='/data/scoresheets/annotations/annotated_sheets_25022019.json'

In [3]:
with open(path, 'r') as f:
    raw_data = json.load(f)
raw_data[5]

{u'Agreement': None,
 u'Created At': u'2018-12-01T10:15:23.000Z',
 u'Created By': u'collaborator.scoresheets@gmail.com',
 u'DataRow ID': u'cjp2001vt4h2o02s6n5vbcsk8',
 u'Dataset Name': u'Scoresheets',
 u'External ID': u'266420df-c93c-49bf-b905-ae906721cce5.jpg',
 u'ID': u'cjp5awkfv4sqe0980z486fdim',
 u'Label': {u'Date': [{u'geometry': [{u'x': 1391, u'y': 0},
     {u'x': 1391, u'y': 67},
     {u'x': 1578, u'y': 67},
     {u'x': 1578, u'y': 0}]}],
  u'Header': [{u'geometry': [{u'x': 1608, u'y': 648},
     {u'x': 60, u'y': 648},
     {u'x': 60, u'y': 0},
     {u'x': 1608, u'y': 0}]}],
  u'Move': [{u'geometry': [{u'x': 164, u'y': 724},
     {u'x': 164, u'y': 774},
     {u'x': 240, u'y': 774},
     {u'x': 240, u'y': 724}]},
   {u'geometry': [{u'x': 634, u'y': 777},
     {u'x': 559, u'y': 777},
     {u'x': 559, u'y': 719},
     {u'x': 634, u'y': 719}]},
   {u'geometry': [{u'x': 153, u'y': 771},
     {u'x': 153, u'y': 828},
     {u'x': 240, u'y': 828},
     {u'x': 240, u'y': 771}]},
   {u'geo

In [10]:
def filter_by_annotator(raw_data):
    filtered_data = []
    for row in raw_data:
        annotator = row["Created By"]
        if annotator.startswith("collaborator"):
            filtered_data.append(row)
    return filtered_data

In [11]:
data = filter_by_annotator(raw_data)

In [12]:
print(len(data))

118


In [27]:
import os
from PIL import Image
def order_box(bbox):
    x1, y1 = bbox["geometry"][0]['x'], bbox["geometry"][0]['y']
    x2, y2 = bbox["geometry"][1]['x'], bbox["geometry"][1]['y']
    x3, y3 = bbox["geometry"][2]['x'], bbox["geometry"][2]['y']
    x4, y4 = bbox["geometry"][3]['x'], bbox["geometry"][3]['y']
    mi_x = min([x1, x2, x3, x4])
    ma_x = max([x1, x2, x3, x4])
    mi_y = min([y1, y2, y3, y4])
    ma_y = max([y1, y2, y3, y4])
    return [mi_x, mi_y, ma_x, mi_y, ma_x, ma_y, mi_x, ma_y]
def create_target_file_east_detector(item, outdir='/data/scoresheets/dataset/'):
    labels = item["Label"]
    item_id = item["External ID"].split(".jpg")[0]
    polygons = []
    for box_type, bboxes in labels.items():
        for bbox in bboxes:
            pg = order_box(bbox)
            polygons.append(pg)
    outfile = os.path.join(outdir, "{}.txt".format(item_id))
    with open(outfile, "w") as out:
        for polygon in polygons:
            polygon = map(str, polygon)
            line = ",".join(polygon) + ",sometext\n"
            out.write(line)
def create_target_file_east_detector_move_section_only(item, outdir='/data/scoresheets/dataset/move-section-only/'):
    try:
        move_section = item["Label"]["Move-Section"][0]
    except:
        print("no move section!!!")
        return None
    move_section_area = order_box(move_section)
    labels = item["Label"]
    item_id = item["External ID"].split(".jpg")[0]
    fp = '/data/scoresheets/dataset/all_annotated_images/' + str(item_id) + '.jpg'
    polygons = []
    for box_type, bboxes in labels.items():
        if box_type in ["Move-Section", "Header"]:
            continue
        for bbox in bboxes:
            pg = order_box(bbox)
            if in_move_section(move_section_area, pg):
                pg = shift_pg(pg, move_section_area)
                polygons.append((box_type, pg))
    outfile = os.path.join(outdir, "{}.txt".format(item_id))
    with open(outfile, "w") as out:
        for t1 in polygons:
            box_type, polygon = t1
            polygon = map(str, polygon)
            line = ",".join(polygon) + ",{}\n".format(box_type.replace(" ", ""))
            out.write(line)
    crop_image(fp, move_section_area)
def crop_image(fp, move_section):
    img = Image.open(fp)
    img = img.crop((move_section[0], move_section[1], move_section[2], move_section[-1]))
    outdir = '/data/scoresheets/dataset/move-section-only/'
    fp_out = outdir + fp.rsplit('/', 1)[-1]
    img.save(fp_out)
def in_move_section(move_section, polygon):
    in_area = False
    if polygon[0] > move_section[0]:
        if polygon[1] > move_section[1]:
            if polygon[2] < move_section[2]:
                if polygon[-1] < move_section[-1]:
                    in_area = True
    return in_area
def shift_pg(pg, move_section):
    dx = move_section[0]
    dy = move_section[1]
    return [pg[0] - dx,
            pg[1] - dy,
            pg[2] - dx,
            pg[3] - dy,
            pg[4] - dx,
            pg[5] - dy,
            pg[6] - dx,
            pg[7] - dy,
           ]
print(create_target_file_east_detector_move_section_only(raw_data[5]))

None


In [28]:
for i, item in enumerate(data):
    print(i, item["External ID"])
    create_target_file_east_detector_move_section_only(item)

(0, u'344786a6-67c9-4e42-8059-c36caa9894fe.jpg')
(1, u'266420df-c93c-49bf-b905-ae906721cce5.jpg')
(2, u'181276fe-2901-4947-b33c-9d3b3a468559.jpg')
(3, u'170506a0-16aa-4776-9bc0-44c1f376b05c.jpg')
(4, u'45770d8d-6438-4fcd-87a7-b0132efa39b0.jpg')
(5, u'45760d64-7ea4-4fb1-8886-22e01dc79fa8.jpg')
(6, u'11935d6f-5aed-4b43-9a69-d465ba39b74a.jpg')
(7, u'1965b710-d13f-454f-9930-db4a0ef62a01.jpg')
(8, u'1918aac4-eb9d-429e-aa3e-28d9de6da4d4.jpg')
(9, u'1612a7b3-9aaf-452d-a1df-777ecef7a912.jpg')
(10, u'1190fd12-b602-4a64-9900-a5d6c0f99292.jpg')
(11, u'465c1196-1ad4-4339-ab4b-a390b5023510.jpg')
(12, u'465a1d7e-bcfa-4caa-a011-2a879cd95841.jpg')
(13, u'459f262f-d51b-4ec1-a8b7-8a722ddf14be.jpg')
(14, u'409ac850-2b75-4715-bc74-ae17f072d9f8.jpg')
(15, u'405e861e-7a65-436c-8e88-7f97bb627766.jpg')
(16, u'372d1b35-fcc1-4bff-bf78-d9ee6e255864.jpg')
(17, u'138cefb9-4280-40af-a453-9fdcf692e754.jpg')
(18, u'102da302-97dc-45ce-829a-ab54fc321eb4.jpg')
(19, u'91c9e124-3f84-41c8-8de4-a2b483dbad6f.jpg')
(20, u'86e